In [152]:
#import necessary packages
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'matplotlib'

In [137]:
#authentication
client = OpenAI(api_key='APIKEY')

In [138]:
#test api call

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role':'system', 'content':'you are a helpful assistant'},
        {'role':'user', 'content':'give me a recipe of a salad dressing that will knock the socks off a vegan?'}
    ]
)

print(response.choices[0].message.content)

Certainly! Here's a delicious recipe for a vegan-friendly salad dressing that is sure to impress:

Creamy Lemon Tahini Dressing

Ingredients:
- 1/4 cup tahini
- Juice of 1 lemon
- 2 tablespoons maple syrup (optional for sweetness)
- 1 clove garlic, minced
- 2 tablespoons water
- Salt and pepper to taste

Instructions:
1. In a mixing bowl, combine the tahini, lemon juice, maple syrup (if using), and minced garlic. Stir well to combine.
2. Slowly add the water while continuously stirring until the dressing reaches your desired consistency. Add more water if you prefer a thinner consistency.
3. Season with salt and pepper to taste.
4. Allow the dressing to sit for a few minutes to let the flavors meld together.
5. Give the dressing a final stir before drizzling it over your favorite salad just before serving. Enjoy!

This creamy lemon tahini dressing adds a tangy and nutty flavor to any salad, making it an excellent choice for vegans and non-vegans alike.


In [47]:
#load in customer data
raw = pd.read_csv('bittext_cx_chat_data.csv')

In [139]:
#sample data and compute tokens
wdf = raw.sample(frac = .01).reset_index(drop=True)
wdf['tokens'] = round(wdf['instruction'].str.split().str.len() * 4/3 + 74, 0)
wdf

,flags,instruction,category,intent,response,tokens
0,BLM,I don't know how I can report a problem with p...,PAYMENT,payment_issue,We understand that reporting a problem with pa...,89.0
1,BL,I need assistance trying to edit my shipping a...,SHIPPING,change_shipping_address,Indeed! I'm here to help you edit your shippin...,86.0
2,BILMQ,where to notify of a trouble with online payments,PAYMENT,payment_issue,I'm attuned to the idea that you're facing tro...,86.0
3,BKM,get invoices from {{Person Name}},INVOICE,get_invoice,I understand your need to access invoices from...,81.0
4,BKLM,edit details on {{Account Category}} account,ACCOUNT,edit_account,How exhilarating it is to embark on this incre...,82.0
...,...,...,...,...,...,...
264,BL,help to see at what time customer service avai...,CONTACT,contact_customer_service,Thank you for reaching out! I completely under...,87.0
265,BIS,is it possible to place an order from {{Delive...,DELIVERY,delivery_options,How enchanting! Your desire to place an order ...,87.0
266,BQ,i dont know how to set a different shipping ad...,SHIPPING,set_up_shipping_address,I can relate to the understanding that you're ...,89.0
267,BL,see purchase {{Order Number}} status,ORDER,track_order,We're here to help! I take note that you would...,81.0


In [140]:
#check stats
categories = wdf['category'].unique()
print(categories)
print('avgtokens ' + str((wdf['tokens'].mean())))
print('ntokens: ' + str(sum(wdf['tokens'])))
print('cost: $' + str(sum(wdf['tokens']) * 0.001 / 1000))

['PAYMENT' 'SHIPPING' 'INVOICE' 'ACCOUNT' 'DELIVERY' 'ORDER' 'REFUND'
 'FEEDBACK' 'CONTACT' 'SUBSCRIPTION' 'CANCEL']
avgtokens 85.80669144981412
ntokens: 23082.0
cost: $0.023082000000000002


In [141]:
#generate a prompt
prompt = 'You are now a simple classifier that takes a ' \
    'string input of a customer serivce chat and gives a single word output in all capital letters.' \
    'These are the following categories that you are allowed to output: ' \
    'CANCEL, ORDER, INVOICE, PAYMENT, ACCOUNT, REFUND, SUBSCRIPTION, DELIVERY, CONTACT. ' \
    'Please summarize the chat query and output one and exactly one word of the above list. ' \
    'Do not output anything more than one word. ' \
    'Here is the customer query: \n'

print('prompt tokens:' + str(len(prompt.split())))
msg = prompt + wdf.at[0, 'instruction']
print(msg)

prompt tokens:74
You are now a simple classifier that takes a string input of a customer serivce chat and gives a single word output in all capital letters.These are the following categories that you are allowed to output: CANCEL, ORDER, INVOICE, PAYMENT, ACCOUNT, REFUND, SUBSCRIPTION, DELIVERY, CONTACT. Please summarize the chat query and output one and exactly one word of the above list. Do not output anything more than one word. Here is the customer query: 
I don't know how I can report a problem with payments


In [142]:
#call chatgpt to get responses
vout = np.array([])
for i in range(len(wdf)):
    
    msg = wdf.at[i, 'instruction'] #gets msg

    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role':'system', 'content': prompt},
            {'role':'user', 'content': msg}
        ]
    )
    
    vout = np.append(vout, response)

    #print(msg)

print(vout)

[ChatCompletion(id='chatcmpl-8aXf37ZXe6SSVKkKb1tPnC4cnduh2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='PAYMENT', role='assistant', function_call=None, tool_calls=None))], created=1703719893, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=121, total_tokens=123))
 ChatCompletion(id='chatcmpl-8aXf4aud32FoUPszpfPypPtS85MnT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='DELIVERY', role='assistant', function_call=None, tool_calls=None))], created=1703719894, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=118, total_tokens=120))
 ChatCompletion(id='chatcmpl-8aXf4OWORE5G0SFyvd72Gy2CMKECd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='PAYMENT', role='a

In [143]:
#get response message from response vector
vresp = np.array([])

for i in range(len(vout)):
    vresp = np.append(vresp, vout[i].choices[0].message.content)

print(vresp)

['PAYMENT' 'DELIVERY' 'PAYMENT' 'INVOICE' 'ACCOUNT' 'DELIVERY' 'ACCOUNT'
 'ACCOUNT' 'ORDER' 'CANCEL' 'INVOICE' 'ACCOUNT' 'DELIVERY' 'INVOICE'
 'DELIVERY' 'CONTACT' 'ORDER' 'ORDER' 'ACCOUNT' 'REFUND' 'CONTACT'
 'REFUND' 'ACCOUNT' 'DELIVERY' 'INVOICE' 'DELIVERY' 'ACCOUNT' 'REFUND'
 'DELIVERY' 'REFUND' 'REFUND' 'DELIVERY' 'CONTACT' 'PAYMENT' 'DELIVERY'
 'DELIVERY' 'DELIVERY' 'ORDER' 'ORDER' 'ORDER' 'REFUND' 'CONTACT'
 'ACCOUNT' 'CONTACT' 'CANCEL' 'INVOICE' 'INVOICE' 'CONTACT' 'CONTACT'
 'ORDER' 'SUBSCRIPTION' 'DELIVERY' 'DELIVERY' 'REFUND' 'PAYMENT' 'ORDER'
 'CONTACT' 'REFUND' 'DELIVERY' 'DELIVERY' 'INVOICE' 'CONTACT' 'DELIVERY'
 'CANCEL' 'PAYMENT' 'ACCOUNT' 'INVOICE' 'CONTACT' 'CANCEL' 'CONTACT'
 'DELIVERY' 'CONTACT' 'INVOICE' 'ACCOUNT' 'ACCOUNT' 'INVOICE' 'CANCEL'
 'CANCEL' 'CONTACT' 'REFUND' 'CONTACT' 'DELIVERY' 'REFUND' 'PAYMENT'
 'DELIVERY' 'DELIVERY' 'DELIVERY' 'ACCOUNT' 'PAYMENT' 'CONTACT' 'REFUND'
 'INVOICE' 'REFUND' 'REFUND' 'PAYMENT' 'UPDATE' 'CONTACT' 'INVOICE'
 'REFUND' 'PAYME

In [144]:
wdf['chatgpt_class'] = vresp
wdf['correct'] = wdf['chatgpt_class'] == wdf['category']
wdf

,flags,instruction,category,intent,response,tokens,chatgpt_class,correct
0,BLM,I don't know how I can report a problem with p...,PAYMENT,payment_issue,We understand that reporting a problem with pa...,89.0,PAYMENT,True
1,BL,I need assistance trying to edit my shipping a...,SHIPPING,change_shipping_address,Indeed! I'm here to help you edit your shippin...,86.0,DELIVERY,False
2,BILMQ,where to notify of a trouble with online payments,PAYMENT,payment_issue,I'm attuned to the idea that you're facing tro...,86.0,PAYMENT,True
3,BKM,get invoices from {{Person Name}},INVOICE,get_invoice,I understand your need to access invoices from...,81.0,INVOICE,True
4,BKLM,edit details on {{Account Category}} account,ACCOUNT,edit_account,How exhilarating it is to embark on this incre...,82.0,ACCOUNT,True
...,...,...,...,...,...,...,...,...
264,BL,help to see at what time customer service avai...,CONTACT,contact_customer_service,Thank you for reaching out! I completely under...,87.0,CONTACT,True
265,BIS,is it possible to place an order from {{Delive...,DELIVERY,delivery_options,How enchanting! Your desire to place an order ...,87.0,ORDER,False
266,BQ,i dont know how to set a different shipping ad...,SHIPPING,set_up_shipping_address,I can relate to the understanding that you're ...,89.0,DELIVERY,False
267,BL,see purchase {{Order Number}} status,ORDER,track_order,We're here to help! I take note that you would...,81.0,ORDER,True


In [147]:
print(
    'accuracy: ' + str(round(len(wdf[wdf['correct'] == True]) / len(wdf) * 100, 1)) + ' %'
)

accuracy: 70.3 %


In [151]:
wdf.to_csv('gpt_classified.csv')